<!-- Table des matières automatique -->
<div style="overflow-y: auto">
  <h1>Table des matières</h1>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

# ACDC 2019-2020 : l'Autre Carré D'à Côté

alias "la zone à JPD"

alias "le plateau de Cournols-Olloix"

(pour faire du DS évidemment)

In [ ]:
%matplotlib inline

In [ ]:
import sys
import os
import math
import re
import datetime as dt
import pandas as pd
import numpy as np

from lxml import etree
import pyproj

import folium
import folium.plugins

import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation

from shapely import geometry

from matplotlib import pyplot as plt
import matplotlib.gridspec as pltg

from IPython.display import HTML

# Communs

In [ ]:
KInfValues = [np.inf, -np.inf]

In [ ]:
kmlNameSpaces = \
  { 'gx' : 'http://www.google.com/kml/ext/2.2',
    'kml' : 'http://www.opengis.net/kml/2.2',
    'atom' : 'http://www.w3.org/2005/Atom' }

In [ ]:
# Projection coordonnées sphériques (degrés)=> coordonnées planes (système cible au choix)
KProjWgs84 = pyproj.Proj(init='epsg:4326') # WGS 84 : long, lat en degrés

KProjUtm31  = pyproj.Proj(init='epsg:32631') # WGS 84 - UTM 31N : long, lat en m
KProjCc46   = pyproj.Proj(init='epsg:3946')  # RGF 93 - CC46    : long, lat en m
KProjLamb93 = pyproj.Proj(init='epsg:2154')  # RGF 93 - Lambert : long, lat en m

def geoProjeter(sCoords, srcProj, tgtProj): # sCoords : [0]=x=long, [1]=y=lat
    return pd.Series(pyproj.transform(srcProj, tgtProj, sCoords[0], sCoords[1]))

# II. Attribution des points aux observateurs 

Grille nettoyée / corrigée (déplacé points hors broussailles, forêt, bocage trop serré, ou trop loin accès ... quand possible ;
 supprimé points en forêt ou innaccessibles cause relief)

In [ ]:
# Version finale.
kmlRoot = etree.ElementTree().parse('ACDC/DS ACDC COURNOLS OLLOIX JPM v4.kml')

## 1) Chargement du polygône définissant la zone couverte

In [ ]:
# Zone à couvrir : On suppose que c'est le 1er polygône
plMark = kmlRoot.find('kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
zonePoly = plMark.find('kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates',
                       namespaces=kmlNameSpaces).text.strip()
dfZonePoly = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in zonePoly.split(' ')],
                          columns=['long', 'lat', 'alt'])
dfZonePoly.head()

## 2) Chargement des points conservés

In [ ]:
def iterPlacemarks(kmlDoc, kmlNameSpaces):
    for pm in kmlDoc.findall('kml:Document/kml:Document/kml:Placemark', namespaces=kmlNameSpaces):
        name = pm.find('kml:name', namespaces=kmlNameSpaces).text
        desc = pm.find('kml:description', namespaces=kmlNameSpaces).text
        #print(name)
        long, lat, alt = pm.find('kml:Point/kml:coordinates', namespaces=kmlNameSpaces).text.split(',')
        yield dict(numero=int(name), description=desc, longitude=float(long), latitude=float(lat), altitude=float(alt))

In [ ]:
dfPoints = pd.DataFrame(data=list(iterPlacemarks(kmlRoot, kmlNameSpaces)))

In [ ]:
# 3) Ajout coordonnées UTM31
dfPoints[['longitude_utm', 'latitude_utm']] = \
  dfPoints[['longitude', 'latitude']].apply(geoProjeter, srcProj=KProjWgs84, tgtProj=KProjUtm31, axis='columns')
dfPoints[['longitude_utm', 'latitude_utm']] = \
  dfPoints[['longitude_utm', 'latitude_utm']].replace(KInfValues, np.nan) / 1000

In [ ]:
dfPoints.set_index('numero', inplace=True)
dfPoints['papier'] = None
dfPoints['naturalist'] = None
dfPoints = dfPoints.reindex(columns=['papier', 'naturalist', 'latitude', 'longitude', 'latitude_utm', 'longitude_utm',
                                     'altitude', 'description'])
len(dfPoints)

In [ ]:
dfPoints.head()

## 4) Attribution aux observateurs

### Contactés, mais pas volontaires pour 2019

#### Liste François, Auvergne ...

* Christian Fargeix 7/3
* Nicole et Christian Taillandier 10/3
* ('Bruno Gilbert', 'lubrubelle@gmail.com', 'non'), # 11/03
* ('Marc Pommarel', 'pommarelmarc@neuf.fr', 'non'), # 11/03
* ('Gérard Lecoz', 'lecozgerard0818@orange.fr', 'non'), # 11/03
* ('Sabine Boursange' , 'sabine.boursange@lpo.fr', 'non') # 13/03
* ('Jean-Jacques Lallemant', 'jj.lallemant@gmail.com', 'non'), # 13/03
* ('Adèle Debaudoin', 'adele.debaudouin@outlook.fr', 'non', 'non', 'non'), # 14/03
* ('Olivier Gimel', 'gimelolivier@orange.fr', 'non', 'non'), # Oui, puis non le 28/03 !
* ('Thyphaine Lyon', 'typhainelyon@gmail.com', 'ok', 'non'), # Oui, puis non le 30/04 !
* ('David Houston', 'd-houston@wanadoo.fr', 'ok', 'non'), # Oui, mais rien fait, pas répondu aux relances
* ('Laurent Maly', 'altla@orange.fr', 'ok', 'non'), # Oui, mais rien fait, pas répondu aux relances

#### Liste glanée sur FA (données sérieuses sur les 3 communes ACDC)

* Luc Souret, luc83@orange.fr, 72 Chemin de Quinson - 83560 St Julien ; préretraire fin 2019, intéressé pour 2020 !
* Paul Nicolas, pbg.nicolas@gmail.com ; pas confiance dans la méthode
* Matthieu Bernard, matthieubernard8944@neuf.fr ; déménage prochainement à Culhat, pas le temps et trop loin

#### Les vrais participants

In [ ]:
# Infos oservateurs : Prénom + Nom, e-mail, statut de volontaire confirmé ?, présence confirmée le 31/03 ?
obseurs = \
[('François Guélin', 'francois.guelin@orange.fr', 'ok', 'oui'),
 ('Jean-Philippe Meuret', 'jpmeuret@free.fr', 'ok', 'oui'),
 ('Sylvain Sainnier', 'sainnier@gmail.com', 'ok', 'oui'), # formation 31/3 et 7 OK
 ('Gilles Saulas', 'gilles.saulas@orange.fr', 'ok', 'non'),
 ('Cyrille Jallageas', 'cyrisle@yahoo.fr', 'ok', 'oui'),
 ('Jean-François Carrias', 'jean-francois.carrias@orange.fr', 'ok', 'non'),
 ('Alex Clamens', 'clamens.alex@wanadoo.fr', 'ok', 'non'),
 ('Anne Citron', 'acitron@orange.fr', 'ok', 'non'),
 ('Sandra Robert', 'sandlise@orange.fr', 'ok', 'oui',),
 ('Camille Fasolin', 'c.fasolin@gmail.com', 'ok', 'non'),
 ('Clément Rollant', 'clement.rollant@lpo.fr', 'ok', 'non'),
 ('Matthieu Clément', 'matthieu.clement@lpo.fr', 'ok', 'oui'),
 ('Thibault Brugerolle', 'tbrugerolle@hotmail.com', 'ok', 'non'),
 ('Pierre Tourret', 'pierre.tourret@wanadoo.fr', 'ok', 'oui'),
 ('Patrick Mougel', 'mougel.patrick@wanadoo.fr', 'ok', 'oui'),
 ('Cyril Brunel', 'cyrilb63@hotmail.fr', 'ok', 'oui'),
 ('Romain Riols', 'romain.riols@lpo.fr', 'ok', 'non'),
 ('Hugo Samain', 'hugo.samain@gmail.com', 'ok', 'non'),
 ('Jean-Pierre Dulphy', 'jp.dulphy@orange.fr', 'ok', 'non')] # Points "superposés" aux siens, 2 procoles en même temps !

dfObseurs = pd.DataFrame(columns=['nom', 'eMail', 'statut', '31 mars'],
                         index=range(1, len(obseurs)+1), data=obseurs)

dfObseurs

In [ ]:
print('Observateurs volontaires :', len(dfObseurs))
print('* Les noms :', ', '.join(dfObseurs.nom))
print('* Les emails :', ', '.join(dfObseurs.eMail))

In [ ]:
df = dfObseurs[dfObseurs['31 mars'] != 'non']
print('Observateurs présents le 31 :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

#### Assignation des points et des protocoles

In [ ]:
# Réinitialisation de toutes les assignations
dfPoints['papier'] = None
dfPoints['naturalist'] = None

In [ ]:
# numPoints : liste des numéros de points
# papier : nom de l'observateur qui inventoriera ces points en mode "DS papier" (défaut : None=personne)
# naturalist  : idem en mode "DS Naturalist" (défaut : None=personne)
def assignerPoints(numPoints, papier=None, naturalist=None):
    
    assert papier is None or papier in list(dfObseurs.nom)
    assert naturalist is None or naturalist in list(dfObseurs.nom)
    if papier:
        assert dfPoints.loc[numPoints, 'papier'].isnull().all()
        dfPoints.loc[numPoints, 'papier'] = papier
    if naturalist:
        assert dfPoints.loc[numPoints, 'naturalist'].isnull().all()
        dfPoints.loc[numPoints, 'naturalist'] = naturalist

In [ ]:
assignerPoints(papier='François Guélin', # OK, papier 02/03
               numPoints=[145, 146, 147, 160, 161, 162,
                          163, 164, 165, 177, 178, 179, # 02/03
                          194, 195, 196, 197, 198, 199, 200,
                          109, 110, 126, 127, 128, 143, 144,  # 03/04
                          180, 181, 229, 216, 232, 233]) # le 04/05 sans garantie

In [ ]:
assignerPoints(naturalist='Jean-Philippe Meuret',
               numPoints=[109, 110, 112, 126, 127, 128, 143, 144,  # OK, naturalist 02/03
                          148, 163, 164, 165, 166, 182, 183, 184, 185]) # Nouvelle série le 30/04

In [ ]:
assignerPoints(papier='Sylvain Sainnier',
               numPoints=[123, 125, 141, 142, 157, 158, 159, 174, 175, 176, 192, 193, # OK, papier 02/03
                         112, 202, 218, 219]) # le 03/05 sans garantie

In [ ]:
assignerPoints(papier='Gilles Saulas',
               numPoints=[262, 263, 280, 281, 282, 283, 284, 299, 300, 301]) # OK, papier 02/03

In [ ]:
assignerPoints(naturalist='Cyrille Jallageas', # OK, naturalist 02/03 et 11/03
               numPoints=[194, 195, 196, 197, 198, # Smartphone
                          199, 200, 201, 202, 218, 219]) # Tablette

In [ ]:
assignerPoints(papier='Jean-François Carrias',
               numPoints=[23, 39, 40, 41, 42, 56, 57, 58, 59, 60]) # OK, papier 02/03

In [ ]:
assignerPoints(papier='Alex Clamens',
               numPoints=[55, 72, 73, 74, 75, 90, 91]) # OK, papier 10/03 (moins le 76 => JPD)

In [ ]:
assignerPoints(papier='Anne Citron',
               numPoints=[210, 211, 212, 228, 245, 246,  # OK, papier 11/03
                          113, 129, 130, # Ajout du 27/04
                          265, 266]) # Ajout du 28/04

In [ ]:
assignerPoints(papier='Camille Fasolin',
               numPoints=[88, 89, 105, 106, 122]) # OK, papier 11/03, tutorat Sylvain et Jean-François

In [ ]:
assignerPoints(naturalist='Matthieu Clément',
               numPoints=[157, 158, 159, 174, 175, 176, 192, 193]) # OK, 12/03

In [ ]:
assignerPoints(papier='Thibault Brugerolle',
               numPoints=[148, 166, 182, 183, 184, 185]) # OK, papier (pb GPS revérifiés, naturalist=KO), 12/03

In [ ]:
assignerPoints(naturalist='Pierre Tourret',
               numPoints=[23, 39, 40, 41, 42, 56, 57, 58, 59, 60]) # ok, 12/03

In [ ]:
assignerPoints(papier='Sandra Robert',
#              numPoints=[230, 231, 232, 233, 249]) # OK, 11/03 (moins 250 => JPD); prévision de 5 points en plus 17/03
               numPoints=[]) # Mais stage, pas sûre de pouvoir du tout               

In [ ]:
assignerPoints(naturalist='Clément Rollant',
               numPoints=[113, 129, 130, 146, 147, 162]) # OK, naturalist

In [ ]:
assignerPoints(naturalist='Cyril Brunel',
               numPoints=[262, 263, 280, 281, 282, 299, 300, 301]) # OK 18/03, naturalist 27/03 ; 283 retiré, pas pu 1er passage

In [ ]:
assignerPoints(naturalist='Romain Riols',
               numPoints=[210, 211, 228, 245, 246, 247, 265, 266, 284, # OK, naturalist, 01/04
                          160, 177, 178, 179, # En plus le 12-13/04, 2ème passage à confirmer
                          180, 181, 215, 216, 232, 233, 250]) # En plus le 01/05, idem ?

In [ ]:
assignerPoints(naturalist='Patrick Mougel', 
               numPoints=[55, 72, 73, 74, 75, 76]) # OK 19/03 naturalist 14/04

In [ ]:
assignerPoints(naturalist='Hugo Samain',
               numPoints=[88, 89, 90, 91, 105, 106, 122, 123,
                          125, 141, 142, 145, 161, # OK 13/04 naturalist
                          212, 213, 229]) # En plus, le 04/05

In [ ]:
assignerPoints(papier='Jean-Pierre Dulphy', # 
               numPoints=[201, 213, 214, 215, 247, # Proches < 70m ceux protocole JPD, pas encore attribués
#                         I2,  H1,  H4,  I1,  J2

                          76, 250]) # Proches < 70m ceux protocole JPD, déjà attribués
#                         B4, I4,

#                         182, 105, 211, # 70m < proches < 150m ceux protocole JPD, pas encore attribués
#                         H1,  C1,  G1,

#                         178, 145, 146, 89, 75, 73, 262&263, 281, # 70m < proches < 150m ceux protocole JPD, déjà attribués.
#                         G2,  D4,  E2,  C2, B1, A1, J1,      J4,

In [ ]:
# Pour tests.
#assignerPoints(papier='Jean-Philippe', numPoints=[23, 39, 40])  # tests
#assignerPoints(naturalist='Jean-Philippe', numPoints=[145, 146, 147])  # tests
#assignerPoints(papier='François', numPoints=[109, 110, 112])  # tests
#assignerPoints(naturalist='François', numPoints=[41, 42])  # tests

In [ ]:
dfPoints.tail()

In [ ]:
# Bilan global
dfBilanAttr = pd.DataFrame(data=[(len(dfPoints[dfPoints.papier == obseur]), len(dfPoints[dfPoints.naturalist == obseur])) \
                             for obseur in dfObseurs.nom] \
                            + [(len(dfPoints[dfPoints.papier.notnull()]), len(dfPoints[dfPoints.naturalist.notnull()])),
                               (len(dfPoints[dfPoints.papier.isnull()]), len(dfPoints[dfPoints.naturalist.isnull()]))],
                       columns=['Papier', 'Naturalist'],
                       index=list(dfObseurs.nom) + ['Assignés', 'Non assignés'])
dfBilanAttr

In [ ]:
# En pourcentages ...
round(100 * dfBilanAttr.loc[['Assignés', 'Non assignés']] / len(dfPoints), 1)

In [ ]:
dict(nbObseursAvecPoints=len(dfBilanAttr[(dfBilanAttr.Papier > 0) | (dfBilanAttr.Naturalist > 0)].index)-2,
     nbObseursBrut=len(dfObseurs))

In [ ]:
# Les points faits en Naturalist mais pas en Papier.
dfPoints[dfPoints.papier.isnull() & dfPoints.naturalist.notnull()].index

In [ ]:
# Les points faits en Papier mais pas en Naturalist.
dfPoints[dfPoints.naturalist.isnull() & dfPoints.papier.notnull()].index

In [ ]:
# Les points pas faits ni en Papier ni en Naturalist.
dfPoints[dfPoints.naturalist.isnull() & dfPoints.papier.isnull()].index

In [ ]:
# Uniquement les observateur présents le 31/03
dfBilanAttr.loc[dfBilanAttr.index.isin(dfObseurs[dfObseurs['31 mars'] != 'non'].nom)]

In [ ]:
df = dfObseurs[dfObseurs.nom.isin(dfBilanAttr.loc[(dfBilanAttr.Papier == 0) & (dfBilanAttr.Naturalist == 0)].index)]
print('Observateurs n\'ayant pas encore choisi leurs points :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

In [ ]:
df = dfObseurs[dfObseurs.nom.isin(dfBilanAttr.loc[(dfBilanAttr.Papier > 0) | (dfBilanAttr.Naturalist == 0)].index)]
print('Observateurs protocole papier (ou encore indécis) :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

In [ ]:
df = dfObseurs[dfObseurs.nom.isin(dfBilanAttr.loc[(dfBilanAttr.Naturalist > 0) | (dfBilanAttr.Papier == 0)].index)]
print('Observateurs protocole naturalist (ou encore indécis) :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

## 5) Export Excel

In [ ]:
attrTableFileName = 'ACDC/ACDC2019-AttributionsPoints.xlsx'

with pd.ExcelWriter(attrTableFileName) as xlWriter:
    dfPoints.reset_index().to_excel(xlWriter, sheet_name='AttribDétails', index=False)
    dfBilanAttr.to_excel(xlWriter, sheet_name='AttribSynthèse')
    dfObseurs.to_excel(xlWriter, sheet_name='Observateurs', index=False)

HTML("""<p>Attributions : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=attrTableFileName))

## 6) Génération des KML individuels, avec cercles d'aide au relevé DS

In [ ]:
#Matrice de couleur des points selon l'assignation papier / naturalist du point
#                               Naturalist    
#                     papier    
KDefPointColors = { True  : { True : skml.Color.red,     False : skml.Color.lightgreen },
                    False : { True : skml.Color.fuchsia, False : skml.Color.cyan } }
# * prefix : for target file name, only used if observer is None
def generateKml(dfPoints, dfZonePoly=None, observer=None, pointsColor=KDefPointColors,
                circles=[(25, 20), (100, 30), (200, 40)], circlesColor=skml.Color.white,
                title=None, prefix=None, postfix='cercles-stoceps', tgtDir='ACDC'):
    
    dfObsverPoints = dfPoints[(dfPoints.papier == observer) | (dfPoints.naturalist == observer)] if observer else dfPoints
    if len(dfObsverPoints) == 0:
        return None
    
    if not isinstance(pointsColor, dict):
        pointsColor = { papAss : { natAss : pointsColor for natAss in [True, False] } for papAss in [True, False] }
    
    kml = skml.Kml(name=title or 'Points ACDC 2019{}'.format(' '+observer if observer else ''))

    labelStyle = { papAss : { natAss : skml.LabelStyle(color=pointsColor[papAss][natAss], scale=1) \
                             for natAss in pointsColor[papAss] } for papAss in pointsColor }
    iconStyle = skml.IconStyle(icon=skml.Icon(href='http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'))
    ptStyle = { papAss : { natAss : skml.Style(labelstyle=labelStyle[papAss][natAss], iconstyle=iconStyle) \
                           for natAss in pointsColor[papAss] } for papAss in pointsColor }
    lineStyle = skml.LineStyle(color=skml.Color.red, width=3)

    if dfZonePoly is not None:
        ls = kml.newlinestring(name='Zone ACDC Cournols-Olloix JPD', extrude=1,
                               coords=dfZonePoly[['long', 'lat', 'alt']].values)
        ls.linestyle = lineStyle
        
    circleStyle = skml.LineStyle(color=circlesColor, width=1)

    obsver = ''
    for idx, sPt in dfObsverPoints.iterrows():
        
        # Point
        pt = kml.newpoint(name=str(idx), coords=[(sPt.longitude, sPt.latitude, 0)], extrude=1)
        pt.style = ptStyle[sPt.papier is not None][sPt.naturalist is not None]

        pt.description = 'Papier: {}, Naturalist: {} - UTM31 : lat={:.3f}km, long={:.3f}km' \
                         .format(sPt.papier or 'Personne', sPt.naturalist or 'Personne',
                                 sPt.longitude_utm, sPt.latitude_utm)
        
        # Cercles concentriques
        for r, n in circles:
            ls = kml.newlinestring(name='rayon={}m'.format(r), extrude=1,
                                   coords=kmlcircle.spoints(long=sPt.longitude, lat=sPt.latitude, meters=r, n=n, offset=0))
            ls.linestyle = circleStyle

    prfx = observer.replace(' ', '').replace('-', '') + '-' if observer else prefix + '-' if prefix else ''
    tgtKmlFileName = os.path.join(tgtDir, 'ACDC2019-{}{}-points{}.kml' \
                                          .format(prfx, len(dfObsverPoints), '-'+postfix if postfix else ''))
    kml.save(tgtKmlFileName)
    
    return tgtKmlFileName

In [ ]:
# Tous les observateurs, 1 par 1, cercles STOC EPS blancs.
html = "<table>"
for indObseur, obseur in enumerate(dfObseurs.nom):
    nomFicCarteAttrib = generateKml(dfPoints, dfZonePoly, obseur)
    html += """<tr><td>{num}</td>
                   <td style='text-align:left'>{obseur}</td>
                   <td  style='text-align:left'>{fileLink}</td></tr>""" \
            .format(num=indObseur+1, obseur=obseur,
                    fileLink="<a href='{fileName}' target='_blank'>{fileName}</a>".format(fileName=nomFicCarteAttrib) \
                             if nomFicCarteAttrib else '')
html += '</table>'

HTML(html)

In [ ]:
# Tous les observateurs, tous ensembles, cercles STOC EPS blancs.
nomFicCarteAttrib = generateKml(dfPoints, dfZonePoly)

HTML("""<p>Tous les points, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCarteAttrib))

In [ ]:
# Tous les observateurs, tous ensembles, cercles 300m éval. milieux fuschia.
nomFicCarteAttrib = generateKml(dfPoints, dfZonePoly, postfix='cercles-milieux', 
                                circles=[(300, 50)], circlesColor=skml.Color.fuchsia)

HTML("""<p>Tous les points, cercles milieux naturels : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCarteAttrib))

In [ ]:
# Un seul observateur, cercles STOC EPS blancs.
#obseur = 'Jean-Philippe Meuret'
#nomFicCarteAttrib = generateKml(dfPoints, dfZonePoly, observer=obseur)
#
#HTML("""<p>Points de {obser}, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
#     .format(obser=obseur, fileName=nomFicCarteAttrib))

In [ ]:
# Tous les points faits en Naturalist, mais pas encore en Papier, cercles STOC EPS blancs.
nomFicCarteAttrib = generateKml(dfPoints[dfPoints.papier.isnull() & dfPoints.naturalist.notnull()], dfZonePoly,
                                prefix='nat-mais-pas-pap')

HTML("""<p>Tous les points, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCarteAttrib))

In [ ]:
raise Exception('On s\'arrête ici !')

## 7) Cartographie des points obtenus

In [ ]:
#tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)
for indPt, sPt in dfPoints.iterrows():
    mrk = folium.Marker(location=(sPt.latitude, sPt.longitude), 
                        popup=folium.Popup('Papier:{}, Naturalist:{} - UTM31: lat={:.3f}km, long={:.3f}km' \
                                           .format(sPt.papier or 'Personne', sPt.naturalist or 'Personne',
                                                   sPt.longitude_utm, sPt.latitude_utm)),
                        icon=folium.Icon(color='green', icon_color='black'))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

# III. Génération du KML pour la matinée de formation du 31/03

In [ ]:
dfPointsForm = pd.DataFrame(data=[dict(papier='testeurs', naturalist=None, latitude=45.64776, longitude=3.04115,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 1'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64752, longitude=3.04500,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 2'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64661, longitude=3.04974,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 3'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64908, longitude=3.04857,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 4')],
                           index=range(1, 5))

In [ ]:
# Tous les points du test, cercles STOC EPS blancs.
nomFicCartePtsForm = genererKml(dfPointsForm, title='ACDC 2019 Points Formation 31/03', postfix='formation-cercles-stoceps')

HTML("""<p>Tous les points, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCartePtsForm))

# IV. Auto-évaluation des compétences des équipes papier et Naturalist

## 1. Chargement et nettoyage (lignes inutiles, espaces qui traînent ...)

In [ ]:
dfAutoEval = pd.read_excel('ACDC/GrilleAutoEvalObservateur-resultats.xlsx', index_col=0).T.reset_index()
#dfAutoEval.head()

In [ ]:
dfAutoEval.drop(columns=[dfAutoEval.columns[i] for i in [1, 4, 5, 9, 11, 12, 14]], inplace=True)
#dfAutoEval.head()

In [ ]:
dColIds = \
{
    'index' : 'obseur',
    'Je connais les chants des 20 espèces les plus communes du secteur ACDC (voir blog ACDC)' : 'ch20Comm',
    'Je pense arriver à détecter l’essentiel des oiseaux chanteurs dans un rayon de 150 m' : 'ch150m',
    'J’ai déjà pratiqué des recensements sur plan où il faut situer les oiseaux' : 'invPlan',
    'J’ai réalisé des points «\xa0 Plateau de Fromages\xa0»' : 'invPlatFrom',
    'J’ai déjà pratiqué des points STOC-EPS' : 'invSTOC',
    '3) mon expérience ornitho' : 'expGenerale',
    '4) Expérience de saisie sur Naturalist (ex\xa0: EPOC)' : 'expNaturalist'
}

dfAutoEval.rename(columns=dColIds, inplace=True)
dfAutoEval.set_index('obseur', drop=True, inplace=True)
dfAutoEval.columns.name = None
dfAutoEval.head()

In [ ]:
dfAutoEval = dfAutoEval.applymap(lambda s: s.strip())

## 2. Conversion numérique des choix

In [ ]:
dChVal = { 'un peu' : 4, 'beaucoup' : 6, 'passionnément' : 8,
           'cliquez pour choisir dans la liste' : np.nan }
dInvVal = { 'jamais' : 0, 'un peu' : 1, 'beaucoup' : 2, 'passionnément' : 3,
            'cliquez pour choisir dans la liste' : np.nan}
dExpGen = { 'moins de 10 ans de terrain régulier' : 2, 'de 10 à 20 ans de terrain régulier' : 4,
            'plus de 20 ans de terrain régulier' : 6, 'je ne sais plus tellement çà fait longtemps' : 6,
            'cliquez pour choisir dans la liste' : np.nan }
dExpNat = { 'EPOC ou formulaire simple, sans positionnement de l’épingle rouge' : 5,
            'EPOC ou formulaire simple, AVEC positionnement de l’épingle rouge' : 10,
            'cliquez pour choisir dans la liste' : np.nan }

In [ ]:
dToRep = { 'ch20Comm' : dChVal, 'ch150m' : dChVal,
           'invPlan' : dInvVal, 'invPlatFrom' : dInvVal, 'invSTOC' : dInvVal,
           'expGenerale' : dExpGen, 'expNaturalist' : dExpNat }
dfAutoEvalNum = dfAutoEval.copy()
dfAutoEvalNum.replace(dToRep, inplace=True)
dfAutoEvalNum.head()

## 3. Calcul des 2 notes individuelles

In [ ]:
dfAutoEvalNum['noteNaturalist'] = dfAutoEvalNum.sum(axis='columns') * 10.0 / 41.0
dfAutoEvalNum['notePapier'] = \
    dfAutoEvalNum[['ch20Comm', 'ch150m', 'invPlan', 'invPlatFrom', 'invSTOC', 'expGenerale']].sum(axis='columns') * 10.0 / 31.0


In [ ]:
dfAutoEvalNum

## 4. Ajout type d'inventaire effectué

In [ ]:
dfAutoEvalNum = dfAutoEvalNum.join(dfBilanAttr)
dfAutoEvalNum.rename(columns={ col : 'nPts'+col for col in ['Papier', 'Naturalist'] }, inplace=True)
dfAutoEvalNum[['nPtsPapier', 'notePapier', 'nPtsNaturalist', 'noteNaturalist']]

## 5. Bilan : note de chaque équipe

### a. Les auto-évaluations manquantes.

In [ ]:
dict(manquants=set(dfObseurs.nom) - set(dfAutoEvalNum.index), recues=len(dfAutoEvalNum), attendues=len(dfObseurs))

### b. Simple moyenne individuelle,

In [ ]:
{ prot : dfAutoEvalNum.loc[dfAutoEvalNum['nPts'+prot] > 0, 'note'+prot].mean() for prot in ['Papier', 'Naturalist'] }

### c. Moyenne individuelle *pondérée* par le nb de points effectués

In [ ]:
def _weighted_average(dfTwoCols):
    return np.average(dfTwoCols[dfTwoCols.columns[0]], weights=dfTwoCols[dfTwoCols.columns[1]])
{ prot : _weighted_average(dfAutoEvalNum.loc[dfAutoEvalNum['nPts'+prot] > 0, ['note'+prot, 'nPts'+prot]]) \
  for prot in ['Papier', 'Naturalist'] }

# V. Stats simples 1er passage Naturalist

## 1. Chargement des exports individuels Faune Auvergne

(exports par observateur, passage 'a' du 1er avril au 15 mai, 'b' du 16 mai au 15 juin)

In [ ]:
# Chargement des exports
dfObsBrutes = pd.DataFrame()
iFic = 1
for nomFic in os.listdir('ACDC'):
    mo = re.match('ACDC2019([ab])-Export(.*).xlsx', nomFic)
    if mo and '.old' not in nomFic:
        dfObsIndiv = pd.read_excel(os.path.join('ACDC', nomFic))
        dfObsIndiv['Passage'] = mo.group(1)
        dfObsBrutes = dfObsBrutes.append(dfObsIndiv, ignore_index=True, sort=False)
        print('#{} ACDC/{} : {} données brutes, total {}'.format(iFic, nomFic, len(dfObsIndiv), len(dfObsBrutes)))
        iFic += 1

#dfObsBrutes.DATE.fillna(value=pd.Timestamp('{}-{}'.format(annee, dateDefaut)), inplace=True) # Date par défaut
dfObsBrutes['Observateur'] = dfObsBrutes[['Nom', 'Prénom']].apply(lambda s: s[1]+' '+s[0], axis='columns')

len(dfObsBrutes)

In [ ]:
dfObsBrutes.columns

In [ ]:
# Sélection des colonnes utiles
colBrutes = ['ID liste', 'Liste complète ?', 'Commentaire de la liste',
             'Passage', 'Date', 'Heure début', 'Heure fin', 'Horaire',
             'Ordre systématique', 'Nom espèce', 'Estimation', 'Nombre', 'Détails',
             'Code atlas', 'Lat (WGS84)', 'Lon (WGS84)', 'Altitude',
             'Lieu-dit', 'Commune', 'Remarque', 'Remarque privée', 'Observateur']
dfObs = dfObsBrutes[colBrutes]

In [ ]:
dfObs.head()

In [ ]:
# Examen des formulaires, pour pister ceux qui nous intéressent, en soupçonnant des erreur de saisie du numéro de point
df = dfObs.loc[(dfObs['ID liste'] > 0)]
len(df), df['ID liste'].nunique(), ', '.join(sorted(df['Commune'].unique()))

In [ ]:
# Les communes concernées (à qq données près probablement)
df = df[(df.Commune.isin(['Ludesse', 'Cournols', 'Olloix', 'Montaigut-le-Blanc']))]
len(df), df['ID liste'].nunique()

In [ ]:
df = df[['ID liste', 'Date', 'Heure début', 'Heure fin', 'Commentaire de la liste', 'Observateur', 'Lieu-dit', 'Commune']]

In [ ]:
dff = df.groupby(['ID liste']).first()

In [ ]:
list(dff['Commentaire de la liste'].unique())

In [ ]:
# Encore des formulaires ACDC sans le commentaire ACDC ?
dff[~dff['Commentaire de la liste'].fillna('').str.contains('acdc', case=False)]

# 2. Filtrage des données : formulaires ACDC

In [ ]:
dfObs = dfObs[(dfObs['ID liste'] != 0) & dfObs['Commentaire de la liste'].str.contains('acdc', case=False)]

len(dfObs)

In [ ]:
dfObs.tail()

In [ ]:
# dfObs['Commentaire de la liste'].unique()

In [ ]:
#mo = re.match('acdc\s*(?:2019\s*){0,1}(?:point\s*){0,1}(\d{2,3})\D*', 'acdc  232 Google', flags=re.IGNORECASE)
#np.nan if not mo else float(mo.group(1))

# 3. Récupération du numéro de point

(dans le commentaire de chaque formulaire)

In [ ]:
(dfObs['Commentaire de la liste'].nunique(), dfObs['Commentaire de la liste'].unique())

In [ ]:
# Nettoyage préalable commentaire liste
KCommList2Rem = { '2019': '', 'point': '', 'pt':' ', 'passage': '',
                 '1er': '', ' 2 pt': '', ' 2 ': ' ', '2ème': '', '2eme': '', 'second': '',
                  '.': ' ', ':': ' ', ',': ' ', '\n': ' ', '/': ' ' }

def cleanupCommListe(commentaire):
    comm = commentaire.lower()
    for k, v in KCommList2Rem.items():
        comm = comm.replace(k, v)
    return comm.strip()

dfObs['Commentaire de la liste'] = dfObs['Commentaire de la liste'].apply(cleanupCommListe)

(dfObs['Commentaire de la liste'].nunique(), dfObs['Commentaire de la liste'].unique())

In [ ]:
KReCommListeNumPt = ['acdc\s+(\d{2,3})\s*.*', '(\d{2,3})\s+acdc\s*.*']

def numeroPointAcdc(commentaire):
    for regExp in KReCommListeNumPt:
        mo = re.match(regExp, commentaire, flags=re.IGNORECASE)
        if mo:
            break
    return np.nan if not mo else float(mo.group(1))

dfObs['Num point ACDC'] = dfObs['Commentaire de la liste'].apply(numeroPointAcdc)

In [ ]:
#numeroPointAcdc('ACDC Point 202. observateur au point.')

In [ ]:
# Nombre et liste des points effectués au moins 1 fois.
dfObs['Num point ACDC'].nunique(), dfObs['Num point ACDC'].unique()

In [ ]:
# Nombre de points sans numéro retrouvé
dfObs[dfObs['Num point ACDC'].isnull()].groupby('Commentaire de la liste').first()

In [ ]:
# Première idée des nbres de données par point et par passage (et des points effectués à chaque passage)
dfObs[['Num point ACDC', 'Passage', 'Commentaire de la liste']].groupby(['Num point ACDC', 'Passage']).count().unstack()

In [ ]:
# Inventaires de Clément, avec vraiment beaucoup de données
dfObs.loc[dfObs['Observateur'] == 'Clément Rollant', ['Date', 'Heure début', 'Nombre']].groupby(['Date', 'Heure début']).size()

In [ ]:
#dfObs.loc[dfObs['Observateur'] == 'Clément Rollant', ['Passage', 'Heure début', 'Nom espèce', 'Nombre']] \
#     .groupby(['Passage', 'Heure début', 'Nom espèce']).sum()

# 4. Suppression des inventaires en trop par passage

(bon, c'est en contrôlant les formulaires en 5. qu'on sait qu'il faut le faire :-)

In [ ]:
# Romain, passage a, points 178 et 179 effectués le 14/4 et le 1er mai : on garde le meilleur passage (à meilleure heure)
sObsAVirer = dfObs['Num point ACDC'].isin([178, 179]) & (dfObs.Date == '2019-04-14')
sObsAVirer.sum()

In [ ]:
dfObs[sObsAVirer].index

In [ ]:
dfObs.drop(labels=dfObs[sObsAVirer].index, inplace=True)

# 5. Contrôle des formulaires

* numéros de point tous bien récupérés (du commentaire liste) ?
* numéros de points récupérés correspondant bien au vrais points inventoriés (cartographie) ?
* points attribués bien inventoriés à chaque passage ?
* 1 seul inventaire par passage ?
* pas de formulaire de plus de 11mn ? (normalement ...)

In [ ]:
dfObs.head()

In [ ]:
# Nombre de points sans numéro retrouvé
dfObs[dfObs['Num point ACDC'].isnull()].groupby('Commentaire de la liste').first()

In [ ]:
# On va devoir savoir si une obs est dans les 5 1ères minues ou pas
dfObs['Heure début'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), s['Heure début']), axis='columns')
dfObs['Heure fin'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), s['Heure fin']), axis='columns')
dfObs['Horaire'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), dt.time.fromisoformat(s['Horaire'])),
                               axis='columns')
dfObs['Minute'] = dfObs['Horaire'] - dfObs['Heure début']

In [ ]:
# Données hors période des formulaires (bizarre, localisées uniquement chez Romain)
#df = dfObs[(dfObs['Horaire'] < dfObs['Heure début']) | (dfObs['Horaire'] > dfObs['Heure fin'])]
#df.to_excel('ACDC2019-DonneesHoraireHorsListe.xlsx', index=False)
#df

In [ ]:
# Tous les formulaires ACDC x passages retenus
dfForms = dfObs[['ID liste', 'Passage', 'Date', 'Heure début', 'Heure fin', 'Num point ACDC', 'Observateur',
                 'Commentaire de la liste', 'Liste complète ?']] \
            .groupby(['ID liste', 'Passage', 'Date', 'Commentaire de la liste']).first()
dfForms['Durée'] = dfForms['Heure fin'] - dfForms['Heure début']
dfForms = dfForms.reset_index().set_index('ID liste').sort_values(by='Observateur')
len(dfForms)

In [ ]:
# Normalement, tous les formulaires ont duré au moins 10mn (à qq s près => >= 9 mn)
dfForms[dfForms['Durée'] < pd.Timedelta(minutes=9)]

In [ ]:
# Normalement, tous les formulaires sont des listes complètes.
dfForms[dfForms['Liste complète ?'] != 1]

In [ ]:
dfForms.drop(columns=['Liste complète ?'], inplace=True)

In [ ]:
dfForms.tail()

In [ ]:
# Numéros de points correctement attribués aux formulaires ?
# 1 seul inventaire par passage et par point ?
dfForms[dfForms.duplicated(subset=['Passage', 'Num point ACDC'], keep=False)]

In [ ]:
# Les doublons de Romain ...
#dfObs.loc[dfObs['Num point ACDC'].isin([178, 179]),
#          ['ID liste', 'Num point ACDC', 'Passage', 'Date', 'Heure début', 'Observateur']] \
#     .groupby(['ID liste', 'Num point ACDC', 'Date', 'Heure début', 'Passage']).count() \
#     .rename(columns=dict(Observateur='Nb données'))

In [ ]:
# Cartographie des données pour vérifier ça ...
obseur = 'Romain Riols'
dfObs2Map = dfObs[dfObs['Observateur'] == obseur]
dfPts2Map = dfPoints[dfPoints.naturalist == obseur]

In [ ]:
#tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

# Polygone limitant à la zone d'étude
poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)

# Les points d'écoute
for indPt, sPt in dfPts2Map.iterrows():
    mrk = folium.Marker(location=(sPt.latitude, sPt.longitude), 
                        popup=folium.Popup('#{} {}'.format(sPt.name, sPt.naturalist or 'Personne')),
                        icon=folium.Icon(color='white', icon_color='black'))
    mrk.add_to(mp)

# Les données, rassemblées par formulaire / point
for formId in dfObs2Map['ID liste'].unique():
    dfObsForm = dfObs2Map[dfObs2Map['ID liste'] == formId]
    numPt = dfObsForm.iloc[0]['Num point ACDC']
    numPt = '???' if pd.isnull(numPt) else int(numPt)
    heure = dfObsForm.iloc[0]['Heure début'].strftime('%H:%M')
    mc = folium.plugins.MarkerCluster(name='#{}/{} {}'.format(numPt, formId, heure),
                                      options=dict(maxClusterRadius=200, disableClusteringAtZoom=16,
                                                   spiderfyOnMaxZoom=False))
    for indObs, sObs in dfObsForm.iterrows():
        sObs = sObs.fillna('')
        eff = sObs['Estimation'] + str(sObs['Nombre']) + ('' if not sObs['Détails'] else ' ({})'.format(sObs['Détails']))
        ca = '' if not sObs['Code atlas'] else ' (c{})'.format(int(sObs['Code atlas']))
        mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                                  radius=8, line_color='#ff0000', fill=True, fill_color='orange', fill_opacity=0.6,
                                  popup=folium.Popup('#{} {} {}-{} : {} {}{}' \
                                                     .format(numPt, heure, sObs['Date'].strftime('%d/%m/%Y'), 
                                                             'xx::yy', #sObs['Horaire'].strftime('%H:%M'),
                                                             eff, sObs['Nom espèce'], ca)))
        mrk.add_to(mc)
    mc.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

In [ ]:
# Formulaires sans numéro de point
dfForms.loc[dfForms['Num point ACDC'].isnull()].drop(columns=['Num point ACDC'])

In [ ]:
dfForms['Num point ACDC'] = dfForms['Num point ACDC'].astype(int)
dfObs['Num point ACDC'] = dfObs['Num point ACDC'].astype(int)

# 6. Suppression des formulaires non conformes

In [ ]:
# Trop courts : Normalement, tous les formulaires ont duré au moins 10mn (à qq s près => >= 9 mn)
dfForms[dfForms['Durée'] < pd.Timedelta(minutes=9)]

In [ ]:
dfForms.drop(53130, inplace=True)

In [ ]:
dfObs.drop(dfObs[dfObs['ID liste'] == 53130].index, inplace=True)

# 7. Bilan formulaires / observateurs

In [ ]:
# Bilan par observateur et par point : listes des points attendus et réalisés, par passage
dfBilanReal = \
    dfPoints.reset_index()[['numero', 'naturalist',]] \
            .rename(columns=dict(numero='Num point ACDC', naturalist='Observateur')) \
        .join(dfForms[['Num point ACDC', 'Observateur', 'Passage']] \
              .set_index(['Num point ACDC', 'Observateur']),
              on=['Num point ACDC', 'Observateur'], how='outer')
len(dfBilanReal)

In [ ]:
# Rappel : les points non attribués (donc pas fait, normal)
list(dfBilanReal.loc[dfBilanReal.Observateur.isnull(), 'Num point ACDC'])

In [ ]:
dfBilanReal = dfBilanReal[dfBilanReal.Observateur.notnull()]
len(dfBilanReal)

In [ ]:
# Les points attribués et pas faits du tout
dfBilanReal[dfBilanReal.Observateur.notnull() & dfBilanReal.Passage.isnull()]

In [ ]:
# Bilan global par observateur : nb de points effectivement inventoriés à chaque passage
dfBilan = dfBilanReal.groupby(['Observateur', 'Passage']).count().rename(columns={'Num point ACDC':'Réalisés'}).unstack()

In [ ]:
# Bilan par observateur : nb de formulaires attendu et réalisé, par passage
dfBilan = dfBilanAttr.loc[dfForms.Observateur.unique(), ['Naturalist']] \
              .join(dfBilan).rename(columns={'Naturalist':'Attendus'})
dfBilan

In [ ]:
dfBilan.sum().to_dict()

In [ ]:
# Répartition des formulaires par durée (résolution : 1mn ; n minutes = dans [n et n+1[ minutes)
dfForms.groupby(['Durée', 'Passage']).size().unstack()

In [ ]:
# Répartition des données par minute d'inventaire (résolution : 1mn ; n minutes = dans [n et n+1[ minutes)
df = dfObs.groupby(['Minute', 'Passage']).size().unstack()
df

In [ ]:
# % de données au delà de la 9ème minute
100 * df[df.index >= pd.Timedelta(minutes=10)].sum() / df.sum()

# 7. Première mise en forme des données en vue de l'exploitation DS

* format des dates,
* code atlas absent => 0,
* extraction des nb de mâles adultes, d'autres adultes et de juvéniles (à partir nbre, code atlas et détails),
* séparation des 5 1ères minutes de chaque formulaire
* suppression des données "en vol"

In [ ]:
dfObs['Détails'].fillna(value='', inplace=True)
dfObs['Code atlas'].fillna(value=0, inplace=True)
dfObs.Date = dfObs.Date.apply(pd.Timestamp)

In [ ]:
dfObs['Détails'].unique()

In [ ]:
# Codes Atlas Biolovision "à 20 valeurs" (les code de Faune Auvergne sont un sous-ensemble)
KCAAucun   =  0 # Pas de code Atlas
KCAObsSimp =  1 # Observation de l'espèce pendant la période de nidification
KCAObsBiot =  2 # Observation de l'espèce pendant la période de nidification dans un biotope adéquat
KCAMalChan =  3 # Mâle chanteur présent en période de nidification, cris nuptiaux ou tambourinage entendus,
                # mâle vu en parade dans un habitat favorable
KCACoupPer =  4 # Couple pendant la période de nidification dans un biotope adéquat
KCACoupTer =  5 # Comportement territorial d'un couple (chant, querelles avec des voisins, etc.)
                # au moins 2 jours a plus d'une semaine d'intervalle dans le même territoire
KCACoupNup =  6 # Comportement nuptial (mâle et femelle observés)
KCAVisNidP =  7 # Visite d'un site de nidification probable
KCACrAlInq =  8 # Cri d'alarme ou de crainte des adultes ou autre comportement agité suggérant la présence
                # d'un nid ou de jeunes aux alentours
KCAPlaqInc =  9 # Plaque incubatrice d'une femelle capturée
KCAConstNd = 10 # Construction d'un nid ou forage d'une cavité
KCAIndSimu = 11 # Oiseau simulant une blessure ou détournant l'attention
KCANidUtil = 12 # Découverte d'un nid ayant été utilisé durant la période de nidification actuelle
KCAJuvDepd = 13 # Jeunes venant de s'envoler (nidicoles) ou poussins en duvet (nidifuges)
KCAAdSitNi = 14 # Adulte gagnant ou quittant un site de nid; comportement révélateur d'un nid occupé
                # dont le contenu ne peut être vérifié (trop haut ou dans une cavité) ou adulte incubant
KCATranFie = 15 # dulte transportant des fientes
KCATranNou = 16 # Adulte transportant de la nourriture pour les jeunes
KCACoqOeuf = 17 # Coquilles d'oeufs éclos (de la période de nidification actuelle)
KCAAdCouve = 18 # Nid avec adulte vu couvant
KCANiOeufs = 19 # Nid avec oeufs
KCAPousNid = 20 # Jeunes au nid vus ou entendus
KCANdfPoss = 30 # Nidification possible
KCANdfProb = 40 # Nidification probable
KCANdfCert = 50 # Nidification certaine
KCAAbsRech = 99 # Espèce absente malgré des recherches

In [ ]:
# Codes atlas et coefficients d'effectifs correspondants
dfCodesAtlas = pd.read_excel('ACDC/VisioNatureCodesAtlas.xlsx', index=0)[['Codes Biolovision', 'Texte FR']]
dfCodesAtlas.set_index('Codes Biolovision', inplace=True)
#dfCodesAtlas.drop([KCAAucun], inplace=True) # Parfois utilisé par erreur ... tant pis on garde
dfCodesAtlas.drop([KCANdfPoss, KCANdfProb, KCANdfCert, KCAAbsRech], inplace=True)
dfCodesAtlas['nMalAd'] = 0.0
dfCodesAtlas.loc[KCAMalChan, 'nMalAd'] = 1
dfCodesAtlas.loc[KCACoupPer, 'nMalAd'] = 0.5
dfCodesAtlas.loc[KCACoupTer, 'nMalAd'] = 0.5
dfCodesAtlas.loc[KCACoupNup, 'nMalAd'] = 0.5
dfCodesAtlas['nJuv'] = 0.0
dfCodesAtlas.loc[KCAJuvDepd, 'nJuv'] = 1
dfCodesAtlas.loc[KCANiOeufs, 'nJuv'] = 1
dfCodesAtlas.loc[KCAPousNid, 'nJuv'] = 1
dfCodesAtlas['nAutAd'] = 1
dfCodesAtlas.loc[KCAAucun, 'nAutAd'] = 1
dfCodesAtlas.loc[KCAMalChan, 'nAutAd'] = 0
dfCodesAtlas.loc[KCACoupPer, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCACoupTer, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCACoupNup, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCAJuvDepd, 'nAutAd'] = 0
dfCodesAtlas.loc[KCANiOeufs, 'nAutAd'] = 0
dfCodesAtlas.loc[KCAPousNid, 'nAutAd'] = 0

assert all(dfCodesAtlas[['nMalAd', 'nAutAd', 'nJuv']].sum(axis='columns') == 1)

dfCodesAtlas

In [ ]:
# Effectif détaillé, par sexe et âge (ad ou juv=pul+1ère année),
# à partir des colonnes ['Nombre', 'Détails', 'Code atlas'],
# supposée formatée ainsi : "<n1>x <sexe> <age> <condition>, ..., <np>x <sexe> <age> <condition>"
# Attention: 
# * Les individus 'en vol' ne sont pas comptés.
# * On conserve quand même les données sans code atlas : certains observateurs les ont beaucoup oubliés !?

KDetCols = ['nMalAd', 'nAutAd', 'nJuv']

KReNumExpect = re.compile(r"([0-9]+)x(.*)")

KEffInvd = pd.Series({ col : np.nan for col in KDetCols })

def calculerEffectifs(sObs, verbose=False):

    # Détails et nbres associés
    effDets = { col : 0 for col in KDetCols }
    
    lstDets = sObs['Détails'].split("/")
    vol = False
    for rawDet in lstDets:
        
        rawDet = rawDet.strip()
        if not rawDet:
            continue
        
        mo = KReNumExpect.match(rawDet)
        if mo:
            
            num = int(mo.group(1))
            sxAgCn = mo.group(2).strip()
            
            sx = 'mal' if 'mâle' in sxAgCn else 'fem' if 'femelle' in sxAgCn else 'alt'
            ag = 'juv' if '1ère année' in sxAgCn or 'poussin' in sxAgCn or 'immature' in sxAgCn \
                       else 'adu' if 'adulte' in sxAgCn or 'année' in sxAgCn or '1 an' in sxAgCn else 'alt'
            cn = 'vol' if 'vol' in sxAgCn else 'alt'
            
            if cn != 'vol':
                
                cat = 'nJuv' if ag == 'juv' else 'nMalAd' if sx == 'mal' else 'nAutAd'
            
                effDets[cat] += num
                
            else:
                
                if len(lstDets) == 1:
                    return KEffInvd
                vol = True
            
        else:
            
            print("Donnée ignorée : Colonne Détails malformée : '{}'".format(rawDet))
            if verbose:
                print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
            
            return KEffInvd
    
    # Code Atlas et Nombre total associé
    codAtls = int(sObs['Code atlas'])
    if codAtls != KCAAucun and codAtls not in dfCodesAtlas.index and not vol:
        print("Donnée ignorée : Code Atlas {} sans intérêt, mais pas en vol : détails={}" \
              .format(codAtls, sObs['Détails']))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
        return KEffInvd

    nbre = sObs['Nombre']
    if codAtls in [KCACoupPer, KCACoupTer, KCACoupNup] and nbre < 2:
        print("Donnée suspecte : Code Atlas pour les couples : {}, mais nb individus < 2 : {} !" \
              .format(codAtls, nbre))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
    
    effNbCode = { colCoefMult : dfCodesAtlas.loc[codAtls, colCoefMult] * nbre \
                 for colCoefMult in KDetCols }
    
    # Quand code atlas indique couple, et que nbre impair d'individus => autre adulte par défaut
    demiMal = effNbCode['nMalAd'] - int(effNbCode['nMalAd'])
    effNbCode['nMalAd'] -= demiMal
    effNbCode['nAutAd'] += demiMal
    
    # Choix final : Priorité aux détails si présents
    if any(effDets.values()):
        if sum(effDets.values()) != sum(effNbCode.values()):
            print("Donnée ignorée : Détails {} et Nombre {} * CodeAtlas {} incohérents : '{}' / '{}'" \
                  .format(rawDet, nbre, codAtls, effDets, effNbCode))
            if verbose:
                print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
            return KEffInvd
        eff = effDets
    else:
        eff = effNbCode
        
    return pd.Series(eff)

In [ ]:
if len(dfObs.Estimation.dropna().unique()) > 0:
    print('Attention : On prend les estimations pour des valeurs exactes.')

In [ ]:
obsCols = ['ID liste', 'Passage', 'Num point ACDC', 'Observateur', 'Date', 'Horaire',
           'Nom espèce', 'Lieu-dit', 'Commune', 'Estimation', 'Nombre', 'Détails', 'Code atlas']

In [ ]:
dfObs[KDetCols] = dfObs[obsCols].apply(calculerEffectifs, axis='columns', verbose=True)
dfObs[KDetCols] = dfObs[KDetCols].fillna(0).astype(int)
dfObs['nDetTot'] = dfObs[KDetCols].sum(axis=1).astype(int)

In [ ]:
# Suppression des données "en vol"
df = dfObs[dfObs['Détails'].str.contains('vol', case=False)]
len(dfObs), len(df) # df

In [ ]:
dfObs.drop(df.index, inplace=True)
len(dfObs)

In [ ]:
#dfObs[dfObs['Code atlas'].isin([4, 5]) & (dfObs.Nombre == 1)].to_excel('ACDC/ACDC2019b-CoupleMais1SeulInd.xlsx', index=False)

In [ ]:
# Extraction des 5 1ères minutes
KDetColsTot = KDetCols + ['nDetTot']

KDetColsTot10 = [c+'10' for c in KDetColsTot]

dfObs.rename(columns=dict(zip(KDetColsTot, KDetColsTot10)), inplace=True)

In [ ]:
KDetColsTot5 = [c+'5' for c in KDetColsTot]
for col in KDetColsTot:
    dfObs[col+'5'] = dfObs.loc[dfObs['Minute'] < pd.Timedelta(minutes=5), col+'10']
    dfObs[col+'5'] = dfObs[col+'5'].fillna(0).astype(int)

In [ ]:
dfObs.sort_values(by=['Num point ACDC', 'Horaire']).head(20)

In [ ]:
# Vérifs
d = dict(donneesAvant5mn=len(dfObs[dfObs['Minute'] < pd.Timedelta(minutes=5)]),
         donneesApres5mn=len(dfObs[dfObs[KDetColsTot5].sum(axis='columns') == 0]),
         total=len(dfObs))
assert d['total'] == d['donneesAvant5mn'] + d['donneesApres5mn']
d

# 6. Premiers contrôles de cohérence des données

In [ ]:
# Cas des absences de code atlas 0 (KCAAucun) hors 'en vol' : des fois justifié, souvent un oubli ?
# (on les a comptés comme code atlas 2)
df = dfObs.loc[(dfObs['Code atlas'] == KCAAucun) & ~dfObs['Détails'].str.contains('vol'), obsCols]
#df.to_excel('ACDC/Codes0.xlsx', index=False)
df

In [ ]:
# Code atlas innatendu dans ce genre d'enquête
dfObs.loc[dfObs['Code atlas'].isin([KCAPlaqInc]), obsCols]

In [ ]:
# Comparaison nb individu total et somme des détails : différence = en vol (mais supprimés => 0)
d = dict(total=dfObs['Nombre'].sum(), detail=dfObs['nDetTot10'].sum(), 
         difference=dfObs['Nombre'].sum() - dfObs['nDetTot10'].sum(),
         en_vol=dfObs.loc[dfObs['Détails'].str.contains('vol'), 'Nombre'].sum())
assert d['difference'] == d['en_vol'], "Incohérence nbre / détails pour les non 'en vol'"
d

In [ ]:
dfObs[obsCols+KDetColsTot5+KDetColsTot10].sort_values(by=['Passage', 'Horaire', 'Num point ACDC']) \
    .to_excel('ACDC/ACDC2019-Naturalist.xlsx', index=False)

# 7. Quelques stats

In [ ]:
# Stats données brutes tous formulaires.
{ 'NbObservateurs' : dfObs.Observateur.nunique(), 
  'NbFormulaires' : dfObs['ID liste'].nunique(),
  'NbJours' : dfObs.Date.nunique(),
  'NbDonnées' : len(dfObs),
  'NbIndividus' : dfObs.Nombre.sum() }

In [ ]:
dfObs['Minute'] = dfObs['Minute'].divide(pd.Timedelta(minutes=1)).astype(int)

In [ ]:
# Effectifs contactés par tranche de temps (5 / 10mn)
dfBilanTemps510 = dfObs[['Num point ACDC', 'Passage', 'nMalAd5', 'nDetTot5', 'nMalAd10', 'nDetTot10']] \
                     .groupby(['Num point ACDC', 'Passage']).sum()
dfBilanTemps510 = \
   dfBilanTemps510.join(dfObs[['Num point ACDC', 'Passage', 'nDetTot10']].rename(columns={'nDetTot10': 'nDonnées10'}) \
                  .groupby(['Num point ACDC', 'Passage']).count())
dfBilanTemps510 = \
   dfBilanTemps510.join(dfObs.loc[dfObs.nDetTot5 > 0, ['Num point ACDC', 'Passage', 'nDetTot5']].rename(columns={'nDetTot5': 'nDonnées5'}) \
                  .groupby(['Num point ACDC', 'Passage']).count())
dfBilanTemps510 = dfBilanTemps510.groupby('Passage').agg(['sum', 'mean'])
#dfBilanTemps510.columns = dfBilanTemps510.columns.swaplevel(0, 1)
dfBilanTemps510 #.sort_values(by=('a', 'nMalAd10'), ascending=False)

In [ ]:
# Effectifs contactés par tranche d'1 minute 
dfBilanTempsDet = dfObs[['Passage', 'Minute', 'nDetTot10']].groupby(['Minute', 'Passage']).sum().unstack()
dfBilanTempsDet.columns = dfBilanTempsDet.columns.levels[1]
_ = dfBilanTempsDet.plot(figsize=(16, 3), marker='.', grid=True,
                         title='Nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
# La même chose ...
_ = dfBilanTempsDet[['a', 'b']].plot(figsize=(16, 4), kind='bar', stacked=False, grid=True,
                                     title='Nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
dfBilanTempsDet

In [ ]:
# Effectifs cumulés contactés par tranche d'1 minute 
for pas in dfBilanTempsDet.columns:
    dfBilanTempsDet['cum({})%'.format(pas)] = 100 * dfBilanTempsDet[pas].cumsum() / dfBilanTempsDet[pas].sum()
_ = dfBilanTempsDet[[c for c in dfBilanTempsDet.columns if 'cum' in c]].plot(figsize=(16, 3), marker='.', grid=True,
                    title='%age cumulé du nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
_ = dfBilanTempsDet[[c for c in dfBilanTempsDet.columns if 'cum' in c]].plot(figsize=(16, 3), grid=True,
                                                                             kind='bar', stacked=False,
                    title='%age cumulé du nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
dfBilanTempsDet

In [ ]:
# Détail par observateur et par passage.
# TODO : 1 graphique par observateur

In [ ]:
# Nbre d'individus contactés par passage et par point, pour les 2 tranches de temps
dfBilanObseursPointDet = dfObs[['Observateur', 'Num point ACDC', 'Passage',
                                'nMalAd5', 'nAutAd5', 'nDetTot5', 'nMalAd10', 'nAutAd10', 'nDetTot10']] \
                           .groupby(['Observateur', 'Num point ACDC', 'Passage']).sum().unstack()
dfBilanObseursPointDet.columns = dfBilanObseursPointDet.columns.swaplevel(0, 1)
dfBilanObseursPointDet.sort_index(axis='columns', inplace=True)
dfBilanObseursPointDet

In [ ]:
dfBilanTempsObseurs = \
   dfObs[['Passage', 'Observateur', 'Minute', 'nMalAd10', 'nAutAd10', 'nDetTot10']] \
     .groupby(['Minute', 'Observateur', 'Passage']).sum().unstack()
dfBilanTempsObseurs.columns = dfBilanTempsObseurs.columns.swaplevel(0, 1)
passages = dfBilanTempsObseurs.columns.levels[0]
for col in dfBilanTempsObseurs.columns.levels[1]:
    dfBilanTempsObseurs[('total', col)] = dfBilanTempsObseurs[[(pas, col) for pas in passages]].sum(axis='columns')
dfBilanTempsObseurs.sort_index(axis='columns', inplace=True)
dfBilanTempsObseurs

In [ ]:
dfBilanTempsObseurs.head()

In [ ]:
# Nbre de données par point par observateur par minute
dfNbPts = pd.DataFrame(index=dfBilanTempsObseurs.index)
dfNbPts.reset_index(inplace=True)
dfNbPts = dfNbPts.join(dfBilan, on='Observateur')
dfNbPts[('Réalisés', 'total')] = sum(dfNbPts[('Réalisés', pas)] for pas in dfBilanTempsObseurs.columns.levels[0][:-1])
dfNbPts.set_index(['Minute', 'Observateur'], inplace=True)

dfBilanTempsObseursMoyParPt = dfBilanTempsObseurs.copy()

for pas in dfBilanTempsObseursMoyParPt.columns.levels[0]:
    for col in dfBilanTempsObseursMoyParPt.columns.levels[1]:
        dfBilanTempsObseursMoyParPt[(pas, col)] = dfBilanTempsObseursMoyParPt[(pas, col)] / dfNbPts[('Réalisés', pas)]
            
dfBilanTempsObseursMoyParPt.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanTempsObseursMoyParPt

In [ ]:
df = dfBilanTempsObseursMoyParPt[('a', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage A : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage A : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = dfBilanTempsObseursMoyParPt[('b', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage B : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage B : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = dfBilanTempsObseursMoyParPt[('total', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='2019 (2 passages) : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='2019 (2 passages) : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
# Rappel nb de points par passage et par observateur : attendu et effectué.
dfBilan

In [ ]:
# Nb de points et de données par observateur
dfBilanObseurs = dfBilan.join(dfObs[['Observateur', 'Horaire', 'Passage']].groupby(['Observateur', 'Passage']) \
                                .count().unstack())
dfBilanObseurs = dfBilanObseurs.join(dfObs[['Observateur', 'nDetTot10', 'nDetTot5', 'nMalAd10', 'nMalAd5', 'nAutAd10', 'nAutAd5', 'Passage']] \
                                     .groupby(['Observateur', 'Passage']).sum().unstack())
dfBilanObseurs.columns = pd.MultiIndex.from_tuples([t if isinstance(t, tuple) else (t, 'total') for t in dfBilanObseurs.columns])
dfBilanObseurs.rename(columns={'Horaire': 'nDonnées'}, level=0, inplace=True)
dfBilanObseurs.columns = dfBilanObseurs.columns.swaplevel(0, 1)

passages = dfBilanObseurs.columns.levels[0][:-1]
for col in dfBilanObseurs.columns.levels[1]:
    if col not in ['Attendus', 'Réalisés']:
        dfBilanObseurs[('total', col)] = dfBilanObseurs[[(pas, col) for pas in passages]].sum(axis='columns')

dfBilanObseurs.sort_index(axis='columns', inplace=True)
dfBilanObseurs.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanObseurs

In [ ]:
# Nbre de données par point par observateur
dfBilanObseursMoyParPt = dfBilanObseurs.copy()

for pas in dfBilanObseursMoyParPt.columns.levels[0]:
    nPtsCol = 'Attendus' if pas == 'total' else 'Réalisés'
    for col in dfBilanObseursMoyParPt.columns.levels[1]:
        if col not in ['Attendus', 'Réalisés']:
            dfBilanObseursMoyParPt[(pas, col)] = dfBilanObseursMoyParPt[(pas, col)] / dfBilanObseursMoyParPt[(pas, nPtsCol)]
            
dfBilanObseursMoyParPt.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanObseursMoyParPt

In [ ]:
# Nb de données / individus par espèce.
dfBilanEspeces = dfObs[['Nom espèce', 'nDetTot10', 'nDetTot5', 'nMalAd10', 'nMalAd5', 'nAutAd10', 'nAutAd5', 'Passage']] \
                .groupby(['Nom espèce', 'Passage']).sum().unstack()
dfBilanEspeces.columns = dfBilanEspeces.columns.swaplevel(0, 1)
dfBilanEspeces.sort_index(axis='columns', inplace=True)

passages = dfBilanEspeces.columns.levels[0]
for col in dfBilanEspeces.columns.levels[1]:
    dfBilanEspeces[('total', col)] = dfBilanEspeces[[(pas, col) for pas in passages]].sum(axis='columns')

dfBilanEspeces.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanEspeces

In [ ]:
# Export Excel
xlsWriter = pd.ExcelWriter('ACDC/ACDC2019-Bilan.xlsx')
dfBilanObseurs.to_excel(xlsWriter, sheet_name='Obseurs')
dfBilanObseursMoyParPt.to_excel(xlsWriter, sheet_name='ObseursParPt')
dfBilanEspeces.to_excel(xlsWriter, sheet_name='Espèces')
dfBilanTemps510.to_excel(xlsWriter, sheet_name='Temps510')
dfBilanTempsDet.to_excel(xlsWriter, sheet_name='TempsDétails')
dfBilanTempsObseurs.to_excel(xlsWriter, sheet_name='TempsObseurs')
dfBilanTempsObseursMoyParPt.to_excel(xlsWriter, sheet_name='TempsObseursParPt')
xlsWriter.save()
xlsWriter.close()

# I. Génération de la grille initiale de points de relevés DS

1. charger KML donnant les limites de la zone
    * lecture KML => polygone zone en coordonnées sphériques
    * conversion / projection en coordonnées métriques : UTM 31
2. générer les points sur la grille et dans les limites de la zone, + certaine distance pour élargir ?
    * grille UTM 31 + qq 100m (qq entier)
    * 1er jet rectancle circonscrit à la zone (xMin, yMin, xMax, yMax) + marge
    * élimination des points hors du polygone de la zone ciblée + marge
3. exporter en KML

Cette grille devra être nettoyée des points inaccessibles, en forêt ...

## 1a) Charger KML donnant les limites de la zone

In [ ]:
kmlRoot = etree.ElementTree().parse('ACDC2019-limites-zone.kml')

In [ ]:
# On suppose que c'est le 1er polygône
plMark = kmlRoot.find('kml:Document/kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
zonePoly = plMark.find('kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates',
                       namespaces=kmlNameSpaces).text.strip()
dfZonePoly = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in zonePoly.split(' ')],
                          columns=['long', 'lat', 'alt'])
dfZonePoly.head()

## 1b) Conversion coords polygône en métriques

In [ ]:
# Attention : Les colonnes sources (x,y) et (x_observateur,y_observateur)
#             sont bizarement des couples (lat, long), et pas l'inverse.
dfZonePoly[['xUtm', 'yUtm']] = \
  dfZonePoly[['long', 'lat']].apply(geoProjeter, srcProj=KProjWgs84, tgtProj=KProjUtm31, axis='columns')
dfZonePoly[['xUtm', 'yUtm']] = dfZonePoly[['xUtm', 'yUtm']].replace(KInfValues, np.nan) #, inplace=True)

In [ ]:
dfZonePoly.head()

In [ ]:
# Rectangle circonscrit à la zone
xUtmMin, yUtmMin, xUtmMax, yUtmMax = \
    dfZonePoly.xUtm.min(), dfZonePoly.yUtm.min(), dfZonePoly.xUtm.max(), dfZonePoly.yUtm.max()
xUtmMin, yUtmMin, xUtmMax, yUtmMax

## 2a) Détermination / choix de la taille des cellules de la grille

In [ ]:
# Le polygone de la zone (et sa surface en ha).
geoZonePoly = geometry.Polygon(shell=[(x, y) for x, y in dfZonePoly[['xUtm', 'yUtm']].itertuples(index=False)])

geoZonePoly.area / 10000

In [ ]:
# Nombre de points approximatif à répartir sur la zone.
nPoints = 150
txCouver = 60 # % ; rapide calcul après avoir dit : 100 points sur 2000 ha de milieux cibles (on vire les forêts)

In [ ]:
# Surface couverte par 1 point
surfPoint = geoZonePoly.area / nPoints
surfPoint / 10000, 'ha'

In [ ]:
# Soit un cercle de diamètre ...
deltaXYPoints = 2 * math.sqrt(surfPoint) * 100 / txCouver / math.pi
deltaXYPoints

In [ ]:
# Bon, on prend plutôt ...
deltaXYPoints = 500 #400

## 2b) Premier jet de points dans rectangle circonscrit + marge d'1 point

In [ ]:
# Rectangle circonscrit + marge d'1 point
xUtmMinR = xUtmMin - deltaXYPoints / 2
xUtmMaxR = xUtmMax + deltaXYPoints / 2
yUtmMinR = yUtmMin - deltaXYPoints / 2
yUtmMaxR = yUtmMax + deltaXYPoints / 2

In [ ]:
# Alignement sur une grille UTM à N m, avec ajustement par décalage en X, Y si besoin
uniteAlign = 100 # m
offsetX = 0
offsetY = 0

xUtmMinR = uniteAlign * math.floor(xUtmMinR / uniteAlign) + offsetX
yUtmMinR = uniteAlign * math.floor(yUtmMinR / uniteAlign) + offsetY
xUtmMaxR = uniteAlign * math.ceil(xUtmMaxR / uniteAlign) + offsetX
yUtmMaxR = uniteAlign * math.ceil(yUtmMaxR / uniteAlign) + offsetY

xUtmMinR, yUtmMinR, xUtmMaxR, yUtmMaxR

In [ ]:
dfPoints = pd.DataFrame(data=[dict(xUtm=x, yUtm=y) \
                              for y in np.arange(yUtmMaxR, yUtmMinR - deltaXYPoints, -deltaXYPoints) \
                              for x in np.arange(xUtmMinR, xUtmMaxR + deltaXYPoints, deltaXYPoints)])
dfPoints['numero'] = range(1, len(dfPoints)+1)
dfPoints[['long', 'lat']] = \
  dfPoints[['xUtm', 'yUtm']].apply(geoProjeter, srcProj=KProjUtm31, tgtProj=KProjWgs84, axis='columns')
dfPoints.set_index('numero', inplace=True)
dfPoints.head()

## 2c) Supprimer les points hors zone + marge

In [ ]:
# Marge en distance au delà de l'appartenance au polygône de la zone.
marginDist = deltaXYPoints * 1.0 #* 0.25

In [ ]:
def pointAroundZone(sXYPoint):
    point = geometry.Point(sXYPoint)
    #return geoZonePoly.contains(point) or point.distance(geoZonePoly) < marginDist
    return point.distance(geoZonePoly) < marginDist # Pas besoin de tester l'appartenance, distance() le fait.
dfPoints['aroundZoneExt'] = \
    dfPoints[['xUtm', 'yUtm']].apply(pointAroundZone, axis='columns')
len(dfPoints), len(dfPoints[dfPoints.aroundZoneExt])

In [ ]:
dfSelPoints = dfPoints[dfPoints.aroundZoneExt]
dfSelPoints.head()

## 3a) Cartographie des points obtenus

In [ ]:
tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
#tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)
for indPt, sPt in dfSelPoints.iterrows():
    mrk = folium.Marker(location=(sPt.lat, sPt.long), 
                        popup=folium.Popup('{} : lat={:.1f}, long={:.1f}'.format(indPt, sPt.lat, sPt.long)),
                        icon=folium.Icon(color='green', icon_color='black'))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

## 3b) Export KML

In [ ]:
dfSelPoints.head()

In [ ]:
kml = skml.Kml(name='Points ACDC 2019 (dist={:.0f}m, marge={:.0f}m, n={})' \
               .format(deltaXYPoints, marginDist, len(dfSelPoints)))

In [ ]:
labelStyle = skml.LabelStyle(color=skml.Color.red, scale=1)
iconStyle = skml.IconStyle(icon=skml.Icon(href='http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'))
ptStyle = skml.Style(labelstyle=labelStyle, iconstyle=iconStyle)

lineStyle = skml.LineStyle(color=skml.Color.red, width=3)

In [ ]:
ls = kml.newlinestring(name='Zone ACDC Cournols-Olloix JPD', extrude=1,
                       coords=dfZonePoly[['long', 'lat', 'alt']].values)
                       #coords=[(long, lat, alt) for long, lat, alt in dfZonePoly[['long', 'lat', 'alt']].itertuples(index=False)])
ls.linestyle = lineStyle

for idx, sPt in dfSelPoints.iterrows():
    pt = kml.newpoint(name=str(idx), coords=[(sPt.long, sPt.lat, 0)], extrude=1)
    pt.style = ptStyle
    pt.description = 'lat={:.1f}, long={:.1f}, alt={:.0f}'.format(sPt.long, sPt.lat, 0)

In [ ]:
tgtKmlFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.kml'.format(len(dfSelPoints), deltaXYPoints, marginDist)
kml.save(tgtKmlFileName)

## 3c) Export Excel

In [ ]:
tgtXlsxFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.xlsx'.format(len(dfSelPoints), deltaXYPoints, marginDist)

dfSelPoints[['xUtm', 'yUtm', 'long', 'lat']].reset_index().to_excel(tgtXlsxFileName, index=False)

# Bac à sable

In [ ]:
# Codes Atlas, par copier + coller de https://wiki.biolovision.net/index.php?title=Correspondance_codes_atlas
dfCodesAtlas = pd.read_csv(io.StringIO("""Codes Biolovision	Codes utilisés en Suisse, Italie et partiellement en France	Codes utilisés en Allemagne et Catalogne	Codes utilisés partiellement en France (EBCC)	Codes utilisés en Autriche	Codes utilisés en Pologne	Texte FR	Texte EN
0	-	-	-	-	-	Pas de code atlas	No atlas code
1	1					Observation de l'espèce pendant la période de nidification	Species observed in breeding season
2	2	A1	1	H	O	Observation de l'espèce pendant la période de nidification dans un biotope adéquat	Species observed in breeding season in possible nesting habitat
3	3	A2	2	S	S	Mâle chanteur présent en période de nidification, cris nuptiaux ou tambourinage entendus, mâle vu en parade dans un habitat favorable	Singing, drumming or displaying male present in breeding season in possible nesting habitat
4	4	B3	3	P	PR	Couple pendant la période de nidification dans un biotope adéquat	Pair (male and female) within safe dates, and in suitable breeding habitat
5	5	B4	4	T	TE	Comportement territorial d'un couple (chant, querelles avec des voisins, etc.) au moins 2 jours a plus d'une semaine d'intervalle dans le même territoire	Territorial behaviour (song, fights with neighbour etc.) on at least two different days a week or more apart at same place indicating a permanently occupied territory
6	6	B5	5	D	KT	Comportement nuptial (mâle et femelle observés)	Courtship behavior (aerial displays, courtship feeding) or copulation
7	7	B6	6	N	OM	Visite d'un site de nidification probable	Visiting probable nest site
8	8	B7	7	A	NP	Cri d'alarme ou de crainte des adultes ou autre comportement agité suggérant la présence d'un nid ou de jeunes aux alentours	Agitated behavior and/or anxiety calls from an adult, suggesting presence of nearby nest or young
9	9	B8	8	I	PL	Plaque incubatrice d'une femelle capturée	Brood patch (Note: code only applies to birds observed in hand and is reserved for experienced birder only)
10	10	B9	9	B	BU	Construction d'un nid ou forage d'une cavité	Nest building observed at nest site (Note: for nest building by wrens, woodpeckers, kingfisher...)
11	11	C10	10	DD	UDA	Oiseau simulant une blessure ou détournant l'attention	Distraction display (especially injury feigning, such as broken wing display) or attacking/dive-bombing humans in defense of unobserved nest or young
12	12	C11a	11	UN	GNS	Découverte d'un nid ayant été utilisé durant la période de nidification actuelle	Used nest (occupied within period of survey); includes inactive nests
13	13	C12	12	FL	MŁO	Jeunes venant de s'envoler (nidicoles) ou poussins en duvet (nidifuges)	Recently fledged young that are incapable of sustained flight
14	14	C13a	13	ON	ZAJ	Adulte gagnant ou quittant un site de nid; comportement révélateur d'un nid occupé dont le contenu ne peut être vérifié (trop haut ou dans une cavité) ou adulte incubant	Occupied nest, but contents not observed; adults entering and remaining for a period of time, then leaving or exchanging duties
15	15	C14a				Adulte transportant des fientes	Adult carrying a fecal sac
16	16	C14b	14	FY	POD	Adulte transportant de la nourriture pour les jeunes	Adult carrying food for young
17	17	C11b		
Coquilles d'oeufs éclos (de la période de nidification actuelle)	Eggshells found (laid within period of survey)
18	18	C13b			WYS	Nid avec adulte vu couvant	Nest with adult incubating
19	19	C15	15	NE	JAJ	Nid avec oeufs	Nest containing eggs
20		C16	16	NY	PIS	Jeunes au nid vus ou entendus	Nest with young seen or heard.
30	30	A	30	30	A	Nidification possible	Possible breeding
40	40	B	40	40	B	Nidification probable	Probable breeding
50	50	C	50	50	C	Nidification certaine	Confirmed breeding
99	99	E99	99	99	NOBS	Espèce absente malgré des recherches	Not observed despite active search
19 codes	20 codes	16 codes	16 codes	17 codes	
"""), sep='\t')

In [ ]:
# Debug extraction nbre / sexe des détails, nbre, code atlas

In [ ]:
verbose = False
sObs = dfObs.loc[2907]
print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))

In [ ]:
print(sObs['Détails'])

# Détails
effDets = { col : 0 for col in KDetCols }

lstDets = sObs['Détails'].split("/")
vol = False
for rawDet in lstDets:

    rawDet = rawDet.strip()
    if not rawDet:
        continue

    mo = KReNumExpect.match(rawDet)
    if mo:

        num = int(mo.group(1))
        sxAgCn = mo.group(2).strip()

        sx = 'mal' if 'mâle' in sxAgCn else 'fem' if 'femelle' in sxAgCn else 'alt'
        ag = 'juv' if '1ère année' in sxAgCn or 'poussin' in sxAgCn or 'immature' in sxAgCn \
                   else 'adu' if 'adulte' in sxAgCn or 'année' in sxAgCn or '1 an' in sxAgCn else 'alt'
        cn = 'vol' if 'vol' in sxAgCn else 'alt'

        if cn != 'vol':

            cat = 'nJuv' if ag == 'juv' else 'nMalAd' if sx == 'mal' else 'nAutAd'

            effDets[cat] += num

        else:

            if len(lstDets) == 1:
                break
            vol = True

    else:

        print("Attention, donnée ignorée : Colonne Détails malformée : '{}'".format(rawDet))
        
vol, effDets, num, sx, ag, cn

In [ ]:
# Code Atlas et Nombre total associé
codAtls = int(sObs['Code atlas'])
if codAtls != 0 and codAtls not in dfCodesAtlas.index and not vol:
    print("Attention, donnée ignorée : Code Atlas {} sans intérêt, mais pas en vol d={}, n={}" \
          .format(codAtls, sObs['Détails'], sObs['Nombre']))
    if verbose:
        print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
    return KEffInvd

nbre = sObs['Nombre']
effNbCode = { colCoefMult : dfCodesAtlas.loc[codAtls, colCoefMult] * nbre \
             for colCoefMult in KDetCols }
if any(effDets.values()):
    if sum(effDets.values()) != sum(effNbCode.values()):
        print("Attention, donnée ignorée : Détails {} et Nombre {} * CodeAtlas {} incohérents : '{}' / '{}'" \
              .format(rawDet, nbre, codAtls, effDets, effNbCode))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
        return KEffInvd
    eff = effDets
else:
    eff = effNbCode

In [ ]:
df = pd.read_excel('ACDC/ACDC2019a-ExportClementRollant.xlsx')
df

In [ ]:
df['Code atlas'].unique()